# An analysis of the State of the Union speeches

**Authors:** Sarah Johnson, Chitwan Kaudan, Nadav Tadelis

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import shelve
import nltk

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = (10, 6)

## Abstract

A [2013 Gaurdian Report](hhttps://www.theguardian.com/world/interactive/2013/feb/12/state-of-the-union-reading-level) used the Flesch-Kincaid Reading Test and found that Presidential speeches and writings have declined in complexity and reading level over time. We decided to test this theory. We analyzed a set of State of the Union Addresses from 1790 to 2017 to analyze the variation in speaking styles amongst US Presidents and analyze how these speaking style evovle over the course of American history. We found that average sentence length and average word length have declined over time. We also found that word length is closely correlated with President's individual speaking styles. So what a President may vary the length of his speech or even the length of his sentences over time, he doesn't tend to vary the size of the words he chooses significantly. Using an MDS analysis we found a clear distinction between words chosen by earlier presidents and words chosen by more modern presidents. Finally, through our sentiment analysis, we discovered that 

..... < add sent or 2 about results from sentiment analysis >
........



## Introduction & Exploring Data Gaps

For our dataset (data/stateoftheunion1970-2017) we used Project Gutenberg's EBook of Complete State of the Union Addresses in which speeches after 2002 were pulled from UCSB The American Presidency Project. 

We first examined our dataset to get a sense of which types of speeches are included, which presidents are represented and if there are significant timeline gaps.

In [4]:
#Load in our data frame of speech titles, president names and dates from notebook 1
addresses = pd.read_hdf('results/df1.h5', 'addresses')

We looked into what types of speeches were included in our dataset. 

In [5]:
addresses['title'].unique()  

array([' State of the Union Address', ' Address on Administration Goals',
       ' Address on Administration Goals (Budget Message)',
       ' Address Before a Joint Session of Congress'], dtype=object)

In [6]:
addresses.loc[addresses['title'] != ' State of the Union Address']

,president,title,date
197,George H.W. Bush,Address on Administration Goals,1989-02-09
201,William J. Clinton,Address on Administration Goals,1993-02-17
209,George W. Bush,Address on Administration Goals (Budget Message),2001-02-27
218,Barack Obama,Address Before a Joint Session of Congress,2009-02-24
226,Donald J. Trump,Address Before a Joint Session of Congress,2017-02-27


Although these speeches are not called State of the Union Addresses a quick Google serach reveals that they were all delivered in front of a Joint Session of Congress around the same time a State of the Union address are typically delivered. They also served the same purpose - they informed the Congress and the public the state of the nation and revealed the administrations top priorities for the coming year. For our we considered these speeches as functionally equivalent to State of Union Adresses.



Next we examined which Presidents were represented and which were not in our dataset.

In [7]:
#We can list out presidents in order and cross reference a complete list of presidents in order to figure out which ones are missing.
pd.Series(addresses['president'].unique(), index = list(range(1, 43)))

1           George Washington
2                  John Adams
3            Thomas Jefferson
4               James Madison
5                James Monroe
6           John Quincy Adams
7              Andrew Jackson
8            Martin van Buren
9                  John Tyler
10                 James Polk
11             Zachary Taylor
12           Millard Fillmore
13            Franklin Pierce
14             James Buchanan
15            Abraham Lincoln
16             Andrew Johnson
17           Ulysses S. Grant
18        Rutherford B. Hayes
19          Chester A. Arthur
20           Grover Cleveland
21          Benjamin Harrison
22           William McKinley
23         Theodore Roosevelt
24            William H. Taft
25             Woodrow Wilson
26             Warren Harding
27            Calvin Coolidge
28             Herbert Hoover
29      Franklin D. Roosevelt
30            Harry S. Truman
31       Dwight D. Eisenhower
32            John F. Kennedy
33          Lyndon B. Johnson
34        

Donald J. Trump is our 45th President yet our dataset only includes 42 Presidents. The Presidents missing from this list are John Tyler and James A. Garfield who both died within a couple days of gaining office so never had a chance to deliver a State of the Union Address. Also, Grover Cleveland served two non-consecutive terms so he was both the 22nd and the 24th president. 

Next we plotted Speech Number agaisnt the date they were delivered to looked at whether there were large timeline gaps in out dataset. 

![Data Gap](fig/timeline.png)

The gap from 1893-1897 represents President Cleveland's second term. A quick google search revealed that President Cleveland did deliver written State of the Union Addresses from 1893-1897 but they are not included in our dataset. Suspecting a formatting error, we took a closer look at all of President Cleveland and found that his 1889 State of the Union ends very unexpectedly mid sentence. It appeared like our data cut off part of his speech and didn;t include his speeches from his second term. We kept these descrepencies in mind when doing our analysis on this speeches. 

## Speech Features Analysis

We started off by caculating certain features for all speeches and looking at the distributions of these features across speeches. Later we examined how these features changed over time and varied across presidents. 

The features we examined were # of sentences, # of characters, # of all words, # of cleaned words which are just all words but with stop words and punctuations removed, # unique cleaned words and lastly, # of unique stemmed cleaned words. Below is a grid of each feature's distribution plot.

![](fig/feature_dists.png)

All feature distributions are bimodal with a large lower peak and small high peak. This told us that most speeches were approximately 4000 cleaned words (or around 200 sentences) but a few speeches were extremely long at 20000 cleaned words (around 1300 sentences). Another thing we noted was that the Unique Cleaned Words and Stemmed Unique Cleaned Words plots had a smaller relative gaps between their peaks. From this we gathered that while there was large range of speech lengths, the distribution of the number of unique words and stemmed uniqued words was much narrower.

Next we examined how speech features changed over time. We expected speech length to increase over time since overtime America became larger so there was more to report on when describing the state of the union. We also know that these addresses over time largely expanded their audiences- they went from written letters to Congress to internally broadcasted speeches. We speculated modern day speeches would be a lot longer since the modern President has to address a diverese set of audience's concerns. 

Since the late 18th century language has become less ornate, so we hypothesized that sentences and words in speeches would probably become shorter with time. This would result in an increased proportion of stop words and punctuations.

And because most of the earlier State of the Union Addresses were written as a letter and not spoken we also expected word diversity or vocabulary size to decline over time as well.

Below is a grid of speech features over time. 

![Speech Changes Over Time](fig/speech_changes.png)

From Log # of Sentences and Log # of Words, we gathered that the eariler presidents from 1790 to about 1820 just have much shorter speeches with fewer sentences and fewer words. Since 1820, speech lengths have fluctuated but remained around the same at approximately 244 sentences and 2,980 words. There doesn't seem to be an obvious increasing or decreasing trend in speech length since the 1840s. 

Just as we expected, both Avg Sentence Length and Avg Word Length graphs show a declining trend over time. Around the late 18th century/early 19th century sentences were around 20-25 words long and today sentences are only 10-12 words long. For words, the change is less drastic, going from 12 characters per word in the 1800s down to 10 characters per word today.

Size of vocabulary dropped significantly from 1790 to around 1842 and sine then has fluctuated but remained steady at a proportion of about 0.4. Speeches from 1900 onwards were spoken rather than written but we don't see a drastic change in vocabulary during this time period. Voculary seems like it depends more on the individual president's style of speaking and less on the method in which the addresses is delivered.

Proportion of Stop Words has slightty fallen over time from a proporiton of 0.55 in the early 1800s to 0.48 today. This is really interesting because even though we correctly predicted sentences length and word length decline, this did not result in a higher proportion of stop words. This might be because addresses went from being written letters which have clear punctuations to spoken transcripts where punctuations are less important. It is also possible that modern Presidents are ending sentences with unconvential stop words.

Next we looked at the same features but time grouped by presidents in order.

![Speech Changes By President](fig/speech_characteristics.png)

Looking at Log # of Sentences and Log # of Words it seems like most presidents tend to a consistent speaking style. Some presidents like Washington tend to give short speeches than other presidents like James Buchanan but in general most presidents speeches are within a 100 sentences or 2500 words of each other. 

Also, while sentence length varies significantly for some presidents, word lenth is fairly consistent for all presidents. It seems like while presidents may vary the length of their speeches or even the length of their sentences, they tend to use the same size words across speeches. Word length seems to be very closely correlated to individual speaking style. 

Presidents Washington, Adams, Jefferson, and Madison have a larger vocabulary size across their speeches as any other president. It seems like around the early 1800s there was change in formality of written or spoken langauge. President Hoover has a huge range of vocabulary size amongst his speeches. Hoover delivered a 1929 State of the Union Address right when the stock market crashed and began the Great Depression. This economic downturn may have greatly affected his speaking manner and how he chose to address a concerned nation.

Also, presidents until Cleveland have a narrow range of stop word proportions. Presidents after Cleveland tend to have a larger range of stop word proportions across speeches of which Hayes and Wilson having the greatest variety. President Cleveland's term was around the time when State of the Union Addresses started being spoken and not written, this may ahve affected the range of stop word proportions for presidents. 


## Working with the data

Before executing further analyses, we needed to find a way to make the data more manegable. This speech data is very rich, and has lots of overlap; if we tried to run analyses using the speech transcripts themselves, it would take far too long. The solution is to manipulate the data to a more efficient structure. To do this we create a frequency matrix `wmat`. Our indices are the unique word stems that we find across all speeches, and our columns are individual speeches. If a cell in row `thank` and column `0` has a value of 4, then we know that the stem `thank` appeared in the first speech 4 times. Clearly this data structure takes up far less memory and is more readily compatible with analysis packages than the richer text data was.

An interesting artifact of the data type is that our frequency matrix is incredibly sparse. In fact, 93% of the individual cells are zeros. This might seem to indicate that this data structure is not the most efficient means of analysing the speeches. However, once we think about the data generating process, the sparsity makes sense. The first large source of sparsity is the exclusion of stopwords, we immedietly remove quite a few words that would be common across all speeches. Once the stopwords are removed, whats left are vocabulary that would readily change from president to president and year to year. The spoken word goes through many iterations even within one presidental term, so expecting large amounts of overlap between speeches might not seem plausible. In this light, the sparsity is not a cause for alarm. In fact, it is an indication that we are extracting only the relevant data from the speech transcripts.

## MDS Analysis

After creating the wmat frequency matrix, we wanted analyze how the types of words used in State of the Union Addresses varied amongst presidents. However, we had a list of 181897 stemmed unique words across all speeches for 42 presidents so performed multi-dimensionality reduction. We used both a Euclidean distance and a Jansen-Shannon divergance to preserve pairwise dissimilarities amongst the datapoints. 

![Euclidean Dist Presidents](fig/mds_naive.png)
![JS Dist Presidents](fig/mds_jsdiv.png)

While the euclidean distance plot shows a rough distinction of earlier presidents on the right and more modern presidents on the left, there are a lot of data points that overlap. Eucleadian distance shows a large disimilarity between presidents that centuries apart (e.g. Washington and Bush) but it does not show much a difference between presidents that are a couple decade apart (ie. Roosevelt and Arthur). 

The Jansen-Shannon divergance provides a much clearer distinction between pre 20th century presidents and post 20th century presidents.

![Euclidean Dist Speeches](fig/mds_naive_all.png)
![Euclidean Dist Speeches](fig/mds_jdsiv_all.png)

## Sentiment Analysis

Next, using our unique non-stemmed words we conducted a sentiment analysis of each State of the Union Address. We looked at the proportion of "positive" words, the proportion of "negative" words, and the ratio of positive to negative words for each speech.

First, we sought to know how sentiments may have changed over time by speech. We found little in the way of clear trends, with quite a bit of variation over time for both positive and negative word usage, but did notice that the positive/negative ratio generally sat at or above 1.0, indicating overall more positive than negative word usage, which is interesting considering that the negative opinion lexicon we used is over twice as large as the positive opinion lexicon (4783 words vs. 2006 words).

We also considered these trends indexed by president rather than by speech, but this looked very similar.

![Sentiment By Speech](fig/sent_time.png)
![Sentiment By President](fig/sent_pres.png)

Next, we were curious as to how sentiment usage might play out over time within political parties (e.g. if a certain party increased their use of negative words over time due to increased focus on fear).

We did not notice clear trends like this among longer-running parties such as Republicans and Democrats, but we did see an interesting rise in positive/negative ratio towards the end of a few former parties' existence (Whig, Federalist, National Union).

![Sentiment By Speech By Party - Unaffiliated](fig/unaffiliated.png)
![Sentiment By Speech By Party - Federalist](fig/federalist.png)
![Sentiment By Speech By Party - Democratic Republican](fig/democrat_repub.png)
![Sentiment By Speech By Party - Democrat](fig/democrat.png)
![Sentiment By Speech By Party - Whig](fig/whig.png)
![Sentiment By Speech By Party - Republican](fig/republican.png)
![Sentiment By Speech By Party - National Union](fig/nat_union.png)

Lastly, we wondered how sentiment might change over time throughout presidencies that presided over large-scale wars. Since the United States has been at war most of its history, we focused our attention on only four wars--the Civil War, World Wars I and II, and the Vietnam War. The first three wars we included due to their size and fatality, and the last we included because of how public opinion of war and the military changed throughout the Vietnam War. We were interested to see whether presiding over a war drastically changed individual presidents' sentiment before, during, and after the wars, as well as how different sentiment among presidents during a particular war may differ, where applicable. It is worth noting that with the exception of World War I, all other wars we considered ended only a few months into the presidency of the last presiding president, due either to the previous president's death (Lincoln, Roosevelt) or resignation (Nixon).

We didn't notice clear trends through either World War I or the Vietnam War, but noted that positive/negative ratio appeared to climb through the Civil War and drop through World War II.

![Sentiment By Speech By War - Civil War](fig/civil_war.png)
![Sentiment By Speech By War - World War I](fig/wwi.png)
![Sentiment By Speech By War - World War II](fig/wwii.png)
![Sentiment By Speech By War - Vietnam War](fig/vietnam_war.png)